In [2]:
import os
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, DepthwiseConv2D, \
    Dense, Concatenate, Add, ReLU, BatchNormalization, AvgPool2D, \
    MaxPool2D, GlobalAvgPool2D, Reshape, Permute, Lambda
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.metrics import categorical_accuracy
from sklearn.model_selection import StratifiedKFold
from tensorflow.keras.metrics import Precision, Recall


def stage(x, channels, repetitions, groups):
    x = shufflenet_block(x, channels=channels, strides=2, groups=groups)
    for i in range(repetitions):
        x = shufflenet_block(x, channels=channels, strides=1, groups=groups)
    return x

def shufflenet_block(tensor, channels, strides, groups):
    x = gconv(tensor, channels=channels // 4, groups=groups)
    x = BatchNormalization()(x)
    x = ReLU()(x)
 
    x = channel_shuffle(x, groups)
    x = DepthwiseConv2D(kernel_size=3, strides=strides, padding='same')(x)
    x = BatchNormalization()(x)
 
    if strides == 2:
        channels = channels - tensor.get_shape().as_list()[-1]
    x = gconv(x, channels=channels, groups=groups)
    x = BatchNormalization()(x)
 
    if strides == 1:
        x = Add()([tensor, x])
    else:
        avg = AvgPool2D(pool_size=3, strides=2, padding='same')(tensor)
        x = Concatenate()([avg, x])
 
    output = ReLU()(x)
    return output

def gconv(tensor, channels, groups):
    input_ch = tensor.get_shape().as_list()[-1]
    group_ch = input_ch // groups
    output_ch = channels // groups
    groups_list = []
 
    for i in range(groups):
        group_tensor = Lambda(lambda x: x[:, :, :, i * group_ch: (i+1) * group_ch])(tensor)
        group_tensor = Conv2D(output_ch, 1)(group_tensor)
        groups_list.append(group_tensor)
 
    output = Concatenate()(groups_list)
    return output

def channel_shuffle(x, groups):  
    _, width, height, channels = x.get_shape().as_list()
    group_ch = channels // groups
 
    x = Reshape([width, height, group_ch, groups])(x)
    x = Permute([1, 2, 4, 3])(x)
    x = Reshape([width, height, channels])(x)
    return x

def build_shufflenet(input_shape, num_classes=2):
    input_tensor = Input(shape=input_shape)
    x = Conv2D(filters=24, kernel_size=3, strides=2, padding='same')(input_tensor)
    x = BatchNormalization()(x)
    x = ReLU()(x)
    x = MaxPool2D(pool_size=3, strides=2, padding='same')(x)

    # repetitions = [3, 7, 3]
    # initial_channels = 384
    # groups = 8
    repetitions = [2, 4, 2]  # Reduced repetitions
    initial_channels = 128  # Reduced initial channels
    groups = 4  # Reduced groups

    for i, reps in enumerate(repetitions):
        channels = initial_channels * (2**i)
        x = stage(x, channels, reps, groups)

    x = GlobalAvgPool2D()(x)
    output = Dense(num_classes, activation='softmax')(x)

    shufflenet_model = Model(inputs=input_tensor, outputs=output)
    return shufflenet_model

# Path to the data directories
fold_data_dir = '/home/wangg/REU-Hearing-Loss-Project/machine learning/allFolds/5folds- 80-20 train test split'
# List all fold directories
all_folds = os.listdir(fold_data_dir)

# K-fold cross-validation
num_folds = 5
skf = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=42)

# Lists to store results for each fold
fold_accuracies = []
fold_precisions = []
fold_recalls = []
fold_f1_scores = []

# Dimensions of the images (224x224 with 3 channels)
img_width, img_height = 224, 224
input_shape = (img_width, img_height, 3)
# Data generator for training and testing (no validation)
data_generator = ImageDataGenerator(rescale=1.0/255.0)

for fold_number in range(1, 6):
    # Create a list of folds excluding the current fold for testing
    test_fold = f'fold{fold_number}'
    train_folds = [f for f in all_folds if f != test_fold]

    # Data generators for training and testing
    train_generator = data_generator.flow_from_directory(
        os.path.join(fold_data_dir, train_folds[0], 'Training'),  # Assuming each fold has the same structure
        target_size=(img_width, img_height),
        # batch_size=32,
        batch_size=64,
        class_mode='categorical',
        shuffle=True
    )

    test_generator = data_generator.flow_from_directory(
        os.path.join(fold_data_dir, test_fold, 'Testing'),
        target_size=(img_width, img_height),
        batch_size=64,
        # batch_size=32,
        class_mode='categorical',
        shuffle=False
    )

    # Build and compile ShuffleNet model
    shufflenet_model = build_shufflenet(input_shape)
    # optimizer = Adam(learning_rate=0.001)
    optimizer = Adam(learning_rate=0.01)  # Increase the learning rate
    loss_function = categorical_crossentropy
    # Add precision and recall to metrics
    metrics = [categorical_accuracy, Precision(), Recall()]
    shufflenet_model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)


    # Train the model
    history = shufflenet_model.fit(
        train_generator,
        steps_per_epoch=train_generator.samples // train_generator.batch_size,
        epochs=60
    )

    # Evaluate the model on test data
    test_loss, test_accuracy, test_precision, test_recall = shufflenet_model.evaluate(test_generator)
    test_f1_score = 2 * (test_precision * test_recall) / (test_precision + test_recall)

    print(f'\nEvaluation for Fold {fold_number + 1}:')
    print("Test Loss:", test_loss)
    print("Test Accuracy:", test_accuracy)
    print("Test Precision:", test_precision)
    print("Test Recall:", test_recall)
    print("Test F1 Score:", test_f1_score)

    # Store the metrics for this fold
    fold_accuracies.append(test_accuracy)
    fold_precisions.append(test_precision)
    fold_recalls.append(test_recall)
    fold_f1_scores.append(test_f1_score)

    # Save the model if needed
    shufflenet_model.save(f'shufflenet_fold_{fold_number + 1}.h5')

# Save results to a file or print as needed
results_filename = 'shufflenet_results.txt'
with open(results_filename, 'w') as file:
    file.write("Fold, Accuracy, Precision, Recall, F1 Score\n")
    for fold_number, (accuracy, precision, recall, f1_score) in enumerate(zip(fold_accuracies, fold_precisions, fold_recalls, fold_f1_scores), start=1):
        file.write(f'{fold_number}, {accuracy}, {precision}, {recall}, {f1_score}\n')

# Print average metrics across folds
average_accuracy = np.mean(fold_accuracies)
average_precision = np.mean(fold_precisions)
average_recall = np.mean(fold_recalls)
average_f1_score = np.mean(fold_f1_scores)

print(f'\nAverage Accuracy across Folds 1-{num_folds}: {average_accuracy}')
print(f'Average Precision across Folds 1-{num_folds}: {average_precision}')
print(f'Average Recall across Folds 1-{num_folds}: {average_recall}')
print(f'Average F1 Score across Folds 1-{num_folds}: {average_f1_score}')

Found 1440 images belonging to 2 classes.
Found 320 images belonging to 2 classes.


Epoch 1/60
22/22 [==============================] - 258s 11s/step - loss: 1.7804 - categorical_accuracy: 0.5291 - precision: 0.5291 - recall: 0.5291
Epoch 2/60
22/22 [==============================] - 285s 13s/step - loss: 0.7106 - categorical_accuracy: 0.6025 - precision: 0.6025 - recall: 0.6025
Epoch 3/60
22/22 [==============================] - 246s 11s/step - loss: 0.6598 - categorical_accuracy: 0.6308 - precision: 0.6308 - recall: 0.6308
Epoch 4/60
22/22 [==============================] - 237s 11s/step - loss: 0.5744 - categorical_accuracy: 0.6962 - precision: 0.6962 - recall: 0.6962
Epoch 5/60
22/22 [==============================] - 262s 12s/step - loss: 0.5696 - categorical_accuracy: 0.7195 - precision: 0.7195 - recall: 0.7195
Epoch 6/60
22/22 [==============================] - 279s 13s/step - loss: 0.6037 - categorical_accuracy: 0.7035 - precision: 0.7035 - recall: 0.7035
Epoch 7/60
22/22 [==============================] - 235s 11s/step - loss: 0.5189 - categorical_accuracy: 0

LookupError: unknown encoding: base64